## DDL

1.Task

CREATE TABLE airline_info (
  airline_id         INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  airline_code       VARCHAR(30)    NOT NULL,
  airline_name       VARCHAR(50)    NOT NULL,
  airline_country    VARCHAR(50)    NOT NULL,
  created_at         TIMESTAMP      NOT NULL DEFAULT NOW(),
  updated_at         TIMESTAMP      NOT NULL DEFAULT NOW(),
  info               VARCHAR(50)    NOT NULL
);

CREATE TABLE airport (
  airport_id         INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  airport_name       VARCHAR(50)    NOT NULL,
  country            VARCHAR(50)    NOT NULL,
  state              VARCHAR(50)    NOT NULL,
  city               VARCHAR(50)    NOT NULL,
  created_at         TIMESTAMP      NOT NULL DEFAULT NOW(),
  updated_at         TIMESTAMP      NOT NULL DEFAULT NOW()
);

CREATE TABLE passengers (
  passenger_id             INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  first_name               VARCHAR(50) NOT NULL,
  last_name                VARCHAR(50) NOT NULL,
  date_of_birth            DATE        NOT NULL,
  gender                   VARCHAR(50) NOT NULL,
  country_of_citizenship   VARCHAR(50) NOT NULL,
  country_of_residence     VARCHAR(50) NOT NULL,
  passport_number          VARCHAR(20) NOT NULL,
  created_at               TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at               TIMESTAMP   NOT NULL DEFAULT NOW()
);

CREATE TABLE flights (
  flight_id            INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  sch_departure_time   TIMESTAMP   NOT NULL,
  sch_arrival_time     TIMESTAMP   NOT NULL,
  departing_airport_id INT         NOT NULL,
  arriving_airport_id  INT         NOT NULL,
  departing_gate       VARCHAR(50) NOT NULL,
  arriving_gate        VARCHAR(50) NOT NULL,
  airline_id           INT         NOT NULL,
  act_departure_time   TIMESTAMP   NOT NULL,
  act_arrival_time     TIMESTAMP   NOT NULL,
  created_at           TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at           TIMESTAMP   NOT NULL DEFAULT NOW()
);

CREATE TABLE booking (
  booking_id       INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  flight_id        INT         NOT NULL,
  passenger_id     INT         NOT NULL,
  booking_platform VARCHAR(50) NOT NULL,
  created_at       TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at       TIMESTAMP   NOT NULL DEFAULT NOW(),
  status           VARCHAR(50) NOT NULL,
  price            DECIMAL(7,2) NOT NULL
);

CREATE TABLE booking_flight (
  booking_flight_id INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  booking_id        INT NOT NULL,
  flight_id         INT NOT NULL,
  created_at        TIMESTAMP NOT NULL DEFAULT NOW(),
  updated_at        TIMESTAMP NOT NULL DEFAULT NOW()
);

CREATE TABLE boarding_pass (
  boarding_pass_id  INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  booking_id        INT         NOT NULL,
  seat              VARCHAR(50) NOT NULL,
  boarding_time     TIMESTAMP   NOT NULL,
  created_at        TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at        TIMESTAMP   NOT NULL DEFAULT NOW()
);

CREATE TABLE baggage (
  baggage_id        INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  weight_in_kg      DECIMAL(4,2) NOT NULL,
  created_at        TIMESTAMP    NOT NULL DEFAULT NOW(),
  updated_at        TIMESTAMP    NOT NULL DEFAULT NOW(),
  booking_id        INT          NOT NULL
);

CREATE TABLE baggage_check (
  baggage_check_id  INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  check_result      VARCHAR(50) NOT NULL,
  created_at        TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at        TIMESTAMP   NOT NULL DEFAULT NOW(),
  booking_id        INT         NOT NULL,
  passenger_id      INT         NOT NULL
);

CREATE TABLE security_check (
  security_check_id INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  check_result      VARCHAR(20) NOT NULL,
  created_at        TIMESTAMP   NOT NULL DEFAULT NOW(),
  updated_at        TIMESTAMP   NOT NULL DEFAULT NOW(),
  passenger_id      INT         NOT NULL
);

2.done

4.done

5.
ALTER TABLE airline_info RENAME TO airline;

6.
ALTER TABLE booking RENAME COLUMN price TO ticket_price;

7.
LTER TABLE flights ALTER COLUMN departing_gate TYPE TEXT;

8.
ALTER TABLE airline DROP COLUMN info;

9.
ALTER TABLE booking
  ADD CONSTRAINT fk_booking_passenger
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE,
  ADD CONSTRAINT fk_booking_flight
    FOREIGN KEY (flight_id) REFERENCES flights(flight_id) ON DELETE CASCADE;

-- booking_flight references booking and flights
ALTER TABLE booking_flight
  ADD CONSTRAINT fk_booking_flight__booking
    FOREIGN KEY (booking_id) REFERENCES booking(booking_id) ON DELETE CASCADE,
  ADD CONSTRAINT fk_booking_flight__flights
    FOREIGN KEY (flight_id) REFERENCES flights(flight_id) ON DELETE CASCADE;

-- boarding_pass references booking
ALTER TABLE boarding_pass
  ADD CONSTRAINT fk_boarding_pass__booking
    FOREIGN KEY (booking_id) REFERENCES booking(booking_id) ON DELETE CASCADE;

-- baggage references booking
ALTER TABLE baggage
  ADD CONSTRAINT fk_baggage__booking
    FOREIGN KEY (booking_id) REFERENCES booking(booking_id) ON DELETE CASCADE;

-- baggage_check references booking and passengers
ALTER TABLE baggage_check
  ADD CONSTRAINT fk_baggage_check__booking
    FOREIGN KEY (booking_id) REFERENCES booking(booking_id) ON DELETE CASCADE,
  ADD CONSTRAINT fk_baggage_check__passenger
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE;

-- security_check references passengers
ALTER TABLE security_check
  ADD CONSTRAINT fk_security_check__passenger
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE;

-- flights references airport (departing/arriving) and airline
ALTER TABLE flights
  ADD CONSTRAINT fk_flights__departing_airport
    FOREIGN KEY (departing_airport_id) REFERENCES airport(airport_id) ON DELETE RESTRICT,
  ADD CONSTRAINT fk_flights__arriving_airport
    FOREIGN KEY (arriving_airport_id) REFERENCES airport(airport_id) ON DELETE RESTRICT,
  ADD CONSTRAINT fk_flights__airline
    FOREIGN KEY (airline_id) REFERENCES airline(airline_id) ON DELETE RESTRICT;

## DML

1.WITH
countries AS (
  SELECT UNNEST(ARRAY[
    'Kazakhstan','Turkey','France','Brazil','Poland','USA','Germany','Spain','Italy','Canada',
    'UK','UAE','Qatar','Japan','Korea','India','China','Australia','Mexico','Netherlands'
  ]) AS country
),
states AS (
  SELECT UNNEST(ARRAY[
    'Almaty Region','Akmola','Ankara','Istanbul','Île‑de‑France','São Paulo','Mazowieckie','Lesser Poland',
    'California','Texas','Bavaria','Catalonia','Lombardy','Ontario','England','Dubai','Doha','Tokyo',
    'Seoul','Delhi','Beijing','NSW','Jalisco','Noord‑Holland'
  ]) AS state
),
cities AS (
  SELECT UNNEST(ARRAY[
    'Almaty','Astana','Ankara','Istanbul','Paris','São Paulo','Warsaw','Kraków',
    'Los Angeles','San Francisco','Munich','Barcelona','Milan','Toronto','London',
    'Dubai','Doha','Tokyo','Seoul','New Delhi','Beijing','Sydney','Guadalajara','Amsterdam'
  ]) AS city
)
INSERT INTO airport (airport_name, country, state, city)
SELECT
  'Airport ' || gs::TEXT AS airport_name,
  (SELECT country FROM countries ORDER BY random() LIMIT 1) AS country,
  (SELECT state   FROM states    ORDER BY random() LIMIT 1) AS state,
  (SELECT city    FROM cities    ORDER BY random() LIMIT 1) AS city
FROM generate_series(1,200) AS gs;

2.
INSERT INTO airline (airline_code, airline_name, airline_country)
VALUES ('KZ','KazAir','Kazakhstan');

3.
UPDATE airline
   SET airline_country = 'Turkey',
       updated_at = NOW()
 WHERE airline_name = 'KazAir';
 

4.
INSERT INTO airline (airline_code, airline_name, airline_country)
VALUES
  ('AE','AirEasy','France'),
  ('FH','FlyHigh','Brazil'),
  ('FF','FlyFly','Poland');

5.
DELETE FROM flights
 WHERE EXTRACT(YEAR FROM act_arrival_time) = 2024
    OR EXTRACT(YEAR FROM sch_arrival_time) = 2024;

6.
UPDATE booking
   SET ticket_price = ROUND(ticket_price * 1.15, 2),
       updated_at = NOW();

7.
DELETE FROM booking
 WHERE ticket_price < 10000;